In [2]:
import pandas as pd
import sqlite3
import os

In [ ]:
data = pd.read_csv('events.csv')

In [ ]:
data.head()

In [ ]:
data['event_time']=pd.to_datetime(data['event_time'])

Seperate rows with event_type = start from event_type = end and create seperate dataframes

In [ ]:
df_start = data[data['event_type'] == 'start'].sort_values(by=['content_ID','device_ID','event_time'])
df_end = data[data['event_type'] == 'end'].sort_values(by=['content_ID','device_ID','event_time'])

In [ ]:
df_start.head()

In [ ]:
df_end.head()

There are equal number of start and end rows

In [ ]:
len(df_start.index)

In [ ]:
len(df_end.index)

In [ ]:
df_start = df_start.reset_index(drop=True)
df_end = df_end.reset_index(drop=True)
df_start['event_id'] = df_start.index
df_start['start_time'] = df_start['event_time']
df_end['end_time']=df_end['event_time']
df_start = df_start.drop(["event_type","event_time"], axis=1)
df_start.head()

For every device_id , content_id combination of start dataframe, the stop dataframe has same device_id and content_id combination. Concatinating these 2 dataframes will give corresponding end_time of that particular event in same file.

In [ ]:
out = (df_start[['device_ID','content_ID']] == df_end[['device_ID','content_ID']]).all(axis=1)
len(df_start[out].index)

In [ ]:
len(df_start)

In [ ]:
df_end = df_end.drop(["event_type","event_time","content_ID","device_ID"],axis=1)
merged = pd.concat([df_start,df_end],axis =1)

All start times are less than end times. The data is consistent.

In [ ]:
len(merged[merged['start_time']< merged['end_time']])
merged.head()

Writing Persons CSV to database

In [ ]:
person = pd.read_csv('persons.csv')
person['id']=person.index
person['appears'] = pd.to_datetime(person['appears'])
person['disappears'] = pd.to_datetime(person['disappears'])
person = person.sort_values(by=['device_id','appears'])
person.head()


In [ ]:
if not os.path.exists('advertima.db'):
    print('Please create a database using create_database.py. \n \
    If advertima.db is in a seperate location, then change the path')

Checking that the tables are created properly

In [ ]:


sql = 'select * from event limit 5;'
sql2 = 'select * from person limit 5;'
cur.execute(sql) 
cur.fetchall()
print("="*100)
cur.execute(sql2)
cur.fetchall()